In [1]:
# paths of the input files
no_head_train_path_0 = '../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../data/phishing/IWSPA-AP-traindata/legit/'
head_train_path_0 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Phish/'
head_train_path_1 = '../data/phishing/Dataset_Full_Header_Training/Dataset_Submit_Legit/'
no_head_test_path = '../data/phishing/IWSPA-APTestData/testdata_noheaders/'
head_test_path = '../data/phishing/IWSPA-APTestData/testdata_fullheaders/'
print('ok')

ok


In [2]:
import os, re, string
import numpy as np
import fasttext
print('ok')

ok


In [3]:
def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text
print('ok')

ok


In [4]:
def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        text_list.append(cleaned_text)
    return text_list, files
print('ok')

ok


In [5]:
no_head_train_0, temp = get_data(no_head_train_path_0)
no_head_train_1, temp = get_data(no_head_train_path_1)
head_train_0, temp = get_data(head_train_path_0)
head_train_1, temp = get_data(head_train_path_1)
no_head_test, no_head_files = get_data(no_head_test_path)
head_test, head_files = get_data(head_test_path)
print('ok')

ok


In [6]:
no_head_train = no_head_train_0 + no_head_train_1
no_head_labels_train = ([0] * len(no_head_train_0)) + ([1] * len(no_head_train_1))

head_train = head_train_0 + head_train_1
head_labels_train = ([0] * len(head_train_0)) + ([1] * len(head_train_1))
print('ok')


ok


## get data statistics

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
print('ok')

ok


In [8]:
tf_vectorizer = CountVectorizer()
X = tf_vectorizer.fit_transform(no_head_train)
print('ok')

ok


In [9]:
print ('#total words', np.matrix.sum(X.todense()))
print ('#unique words',len(set(tf_vectorizer.get_feature_names())))
print('ok')

#total words 1101893
#unique words 82075
ok


In [10]:
tf_vectorizer = CountVectorizer(head_train)
X = tf_vectorizer.fit_transform(head_train)
print('ok')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ok


In [11]:
print ('#total words', np.matrix.sum(X.todense()))
print ('#unique words',len(set(tf_vectorizer.get_feature_names())))
print('ok')

#total words 2065327
#unique words 117085
ok


## without header

In [12]:
shuffled_indices = np.random.permutation(len(no_head_labels_train))
train_data = np.array(no_head_train)[shuffled_indices]
train_data = train_data.tolist()
train_label = np.array(no_head_labels_train)[shuffled_indices]
train_label = train_label.tolist()
print('ok')

ok


In [13]:
temp_train_data = train_data[0:int(0.8*len(train_data))]
temp_train_label = train_label[0:int(0.8*len(train_label))]
temp_test_data = train_data[int(0.8*len(train_data)):]
temp_test_labels = train_label[int(0.8*len(train_label)):]
print('ok')

ok


In [14]:
fast_train_file = './.data/fast_train.txt'
fast_test_file = './.data/fast_test.txt'
writeFile = open(fast_train_file, 'w')
for text, label in zip(temp_train_data, temp_train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()

writeFile = open(fast_test_file, 'w')
for text, label in zip(temp_test_data, temp_test_labels):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [15]:
classifier = fasttext.supervised(fast_train_file, './.models/Amrita-NLP_TOP_fastText_noheaders')
print('ok')

ok


In [16]:
result = classifier.test(fast_test_file)
print('ok')

ok


In [17]:
result.precision, result.recall, result.nexamples

(0.9676855895196507, 0.9676855895196507, 1145)

In [18]:
print (classifier.min_count, classifier.dim, classifier.epoch, classifier.word_ngrams, classifier.encoding, classifier.loss_name, classifier.maxn, classifier.t)

1 100 5 1 utf-8 softmax 0 0.0001


In [19]:
final_fast_train_file = './.data/final_fast_train.txt'
writeFile = open(final_fast_train_file, 'w')
for text, label in zip(train_data, train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [20]:
classifier = fasttext.supervised(final_fast_train_file, './.models/Amrita-NLP_TOP_fastText_noheaders')
print('ok')

ok


In [21]:
len(no_head_test)

4300

In [22]:
for i in range (len(no_head_test)):
    if len(no_head_test[i]) == 0:
        no_head_test[i] = '  '
print('ok')

ok


In [23]:
final_labels = classifier.predict(no_head_test)
print('ok')

ok


In [24]:
writeFile = open('./.submission/Amrita-NLP_submission_TOP_noheaders_1.txt', 'w')
for value, test_file in zip(final_labels,no_head_files):
    writeFile.write(test_file + ' ' + value[0])
    writeFile.write('\n')
writeFile.close()
print('ok')

ok


## with header

In [25]:
shuffled_indices = np.random.permutation(len(head_labels_train))
train_data = np.array(head_train)[shuffled_indices]
train_data = train_data.tolist()
train_label = np.array(head_labels_train)[shuffled_indices]
train_label = train_label.tolist()
print('ok')

ok


In [26]:
temp_train_data = train_data[0:int(0.8*len(train_data))]
temp_train_label = train_label[0:int(0.8*len(train_label))]
temp_test_data = train_data[int(0.8*len(train_data)):]
temp_test_labels = train_label[int(0.8*len(train_label)):]
print('ok')

ok


In [27]:
fast_train_file = './.data/fast_train.txt'
fast_test_file = './.data/fast_test.txt'
writeFile = open(fast_train_file, 'w')
for text, label in zip(temp_train_data, temp_train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()

writeFile = open(fast_test_file, 'w')
for text, label in zip(temp_test_data, temp_test_labels):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [28]:
classifier = fasttext.supervised(fast_train_file, './.models/Amrita-NLP_TOP_fastText_headers')
print('ok')

ok


In [29]:
result = classifier.test(fast_test_file)

In [30]:
result.precision, result.recall, result.nexamples

(0.9923664122137404, 0.9923664122137404, 917)

In [31]:
final_fast_train_file = './.data/final_fast_train.txt'
writeFile = open(final_fast_train_file, 'w')
for text, label in zip(train_data, train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [32]:
classifier = fasttext.supervised(final_fast_train_file, './.models/Amrita-NLP_TOP_fastText_headers')
print('ok')

ok


In [33]:
final_labels = classifier.predict(head_test)
print('ok')

ok


In [34]:
writeFile = open('./.submission/Amrita-NLP_submission_TOP_headers_1.txt', 'w')
for value, test_file in zip(final_labels, head_files):
    writeFile.write(test_file + ' ' + value[0])
    writeFile.write('\n')
writeFile.close()
print('ok')

ok


## combined model

In [35]:
no_head_train = no_head_train_0 + no_head_train_1
no_head_labels_train = ([0] * len(no_head_train_0)) + ([1] * len(no_head_train_1))

head_train = head_train_0 + head_train_1
head_labels_train = ([0] * len(head_train_0)) + ([1] * len(head_train_1))

temp_train = no_head_train + head_train
temp_labels = no_head_labels_train + head_labels_train
print('ok')

ok


In [36]:
shuffled_indices = np.random.permutation(len(temp_labels))
train_data = np.array(temp_train)[shuffled_indices]
train_data = train_data.tolist()
train_label = np.array(temp_labels)[shuffled_indices]
train_label = train_label.tolist()
print('ok')

ok


In [37]:
temp_train_data = train_data[0:int(0.8*len(train_data))]
temp_train_label = train_label[0:int(0.8*len(train_label))]
temp_test_data = train_data[int(0.8*len(train_data)):]
temp_test_labels = train_label[int(0.8*len(train_label)):]
print('ok')

ok


In [38]:
fast_train_file = './.data/fast_train.txt'
fast_test_file = './.data/fast_test.txt'
writeFile = open(fast_train_file, 'w')
for text, label in zip(temp_train_data, temp_train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()

writeFile = open(fast_test_file, 'w')
for text, label in zip(temp_test_data, temp_test_labels):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [39]:
classifier = fasttext.supervised(fast_train_file, './.models/model_combined')
print('ok')

ok


In [40]:
result = classifier.test(fast_test_file)

In [41]:
result.precision, result.recall, result.nexamples

(0.983996120271581, 0.983996120271581, 2062)

In [42]:
final_fast_train_file = './.data/final_fast_train.txt'
writeFile = open(final_fast_train_file, 'w')
for text, label in zip(train_data, train_label):
    writeFile.write('__label__'+str(label)+' '+str(text.encode('utf-8'))+'\n')
writeFile.close()
print('ok')

ok


In [43]:
classifier = fasttext.supervised(final_fast_train_file, './.models/model_combined')
print('ok')

ok


In [44]:
for i in range (len(no_head_test)):
    if len(no_head_test[i]) == 0:
        no_head_test[i] = '  '
print('ok')

ok


In [45]:
final_head_labels = classifier.predict(head_test)

In [46]:
final_no_head_labels = classifier.predict(no_head_test)

In [47]:
writeFile = open('./.submission/Amrita-NLP_submission_headers_2.txt', 'w')
for value, test_file in zip(final_head_labels , head_files):
    writeFile.write(test_file + ' ' + value[0])
    writeFile.write('\n')
writeFile.close()
print('ok')

ok


In [48]:
writeFile = open('./.submission/Amrita-NLP_submission_noheaders_2.txt', 'w')
for value, test_file in zip(final_no_head_labels,no_head_files):
    writeFile.write(test_file + ' ' + value[0])
    writeFile.write('\n')
writeFile.close()
print('ok')

ok
